In [115]:
#@title
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback

time: 856 µs (started: 2023-06-19 11:29:26 +00:00)


In [ ]:
#@title
#agar train karna ho toh
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback


# Read data
data = df_news.copy()
# data = data.sample(frac = 0.1)
# Define pretrained tokenizer and model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=k, output_hidden_states=True)

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["content"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average = 'weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
class SaveModelCallback(EarlyStoppingCallback):
    def __init__(self, output_dir):
        super().__init__(early_stopping_patience=3)
        self.output_dir = output_dir

    def on_epoch_end(self, args, state, control, **kwargs):
        super().on_epoch_end(args, state, control, **kwargs)
        epoch = state.epoch
        model = control.model  # Access the model directly from the Trainer object
        model.save_pretrained(os.path.join(self.output_dir, f"checkpoint-{epoch}"))
# ... Rest of your code ...

args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end = True

)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

In [ ]:
model_path = "output/checkpoint-12000"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=10,output_hidden_states=True)
tokenizer = RobertaTokenizer.from_pretrained(model_path)

In [ ]:
def get_embedding(model, tokenizer, text):
    model.eval()

    with torch.no_grad():
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs.hidden_states
        last_layer_hidden_states = hidden_states[-1]
        embeddings = torch.mean(last_layer_hidden_states, dim=1).squeeze().cpu().numpy()

    return embeddings

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
# Compute document embeddings for all articles in the dataset
document_embeddings = {}

df = df_news.copy()

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    newsid = row['hid']
    text = row['content']

    document_embeddings[newsid] = get_embedding(model, tokenizer, text)

In [1]:
pwd

'/'

# Regressor 

In [116]:



import datetime
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
print(0)
import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle
import sys
import os

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"

#conf = SparkConf().setAll([('spark.driver.memory', '20g'), ('spark.broadcast.blockSize', '10m'), ("spark.executor.instances", '30')])
#sc = SparkContext(conf=conf)
#sqlContext = SQLContext(sc)

conf = SparkConf().setAll([('spark.sql.broadcastTimeout',1000)
    # ('spark.driver.memory', '20g'), \
#                            ('spark.broadcast.blockSize', '10m'),\
#                            ('spark.dynamicAllocation.enabled','False'),\
#                            ('spark.executor.instances','9999')
# #                            ,("spark.sql.autoBroadcastJoinThreshold",-1)
                          ])
sc = SparkContext.getOrCreate(conf=conf)
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
0
time: 17.7 ms (started: 2023-06-19 11:29:44 +00:00)


In [117]:
conf.set("spark.driver.maxResultSize", "100g")

time: 2.46 ms (started: 2023-06-19 11:29:52 +00:00)


In [118]:
def get_path(dates, prefix, padding=None):
    dates_ = dates + []
    if padding:
        st_date, ed_date = sorted(dates)[0], sorted(dates)[-1]
        for i in range(1, 4):
            d = (datetime.datetime.strptime(ed_date, date_fmt) + datetime.timedelta(days=i)).strftime(date_fmt)
            if d < datetime.datetime.today().strftime(date_fmt):
                dates_.append(d)
    dates_ = list(set(dates_) - set(["2019/02/17", "2019/02/18", "2019/05/28", "2019/06/03", "2019/07/02", "2019/07/03", "2019/07/04", "2019/11/13", "2019/11/14", "2020/02/22", "2020/03/31", "2020/04/16", "2020/04/18", "2020/05/11", "2021/05/13"]))
    paths = []
    for date in dates_:
        base_path = NIS_DATA_BASE_PATH
        if date < "2018/06/26":
            base_path = NIS_OLD_DATA_BASE_PATH
        paths.append(base_path + date + "/" + prefix + "/*.parquet")
    return paths

date_fmt = "%Y/%m/%d"
month_fmt = "%Y/%m"

def millis2date(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(date_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(date_fmt)
    except:
        return "1970/01/01"

def millis2month(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(month_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(month_fmt)
    except:
        return "1970/01"

millis2date_udf = F.udf(millis2date, StringType())
millis2month_udf = F.udf(millis2month, StringType())

def divide_maps(d1, d2):
    keys = set(d1.keys()).intersection(set(d2.keys()))
    res = {}
    for k in keys:
        res[k] = d1[k] * 1. / (d2[k] + 1e-10)
    return res

def timediff(y, x, date_fmt="%Y/%m/%d"): 
    end = datetime.datetime.strptime(y, date_fmt)
    start = datetime.datetime.strptime(x, date_fmt)
    delta = (end - start).days
    return delta

def monthdiff(y, x, month_fmt="%Y/%m"): 
    millis = y - x
    delta = millis / (1000 * 3600 * 24 * 30)
    return delta

timediff_udf = udf(timediff, IntegerType())
monthdiff_udf = udf(monthdiff, IntegerType())

def filter_platform(data, platform=None):
    if platform == "ANDROID":
        data = data.filter(data.platform == "ANDROID")
    elif platform == "IOS":
        data = data.filter(data.platform != "ANDROID")
    return data

def filter_category(data, categories=None):
    if categories:
        data = data.filter(data.categoryWhenEventHappened.isin(categories))
    return data

def filter_tenant(data, tenant=None):
    if tenant in ['hi', 'HINDI']:
        data = data.filter(data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    elif tenant in ['en', 'ENGLISH']:
        data = data.filter(~data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    return data


def filter_app(data, app_name=None):
    if 'appName' in data.columns:
        if app_name:
            data = data.filter(data.appName == app_name)
        else:
            data = data.filter((data.appName != "mini") & (data.appName != "crux"))
    return data


time: 3.57 ms (started: 2023-06-19 11:29:55 +00:00)


In [119]:
def get_raw_path(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('deviceId' in x) and (x['deviceId'] != '')
        time_filter = ('timeSpent' in x) and (int(x['timeSpent']) <= 100) and (int(x['timeSpent']) >= 0)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths)             .map(json.loads)             .filter(lambda x: "batch" in x).flatMap(lambda x: x["batch"])             .filter(lambda x: ("event" in x) and (x["event"].lower() == "timespent-front"))             .filter(view_data_filters)             .map(lambda x: x['properties'])
        view_data = rdd.map(lambda x: (x['deviceId'], x['hashId'][:-2], x['timeSpent'],x['Notification Shown'],x['Notification Open'])).toDF(['deviceId', 'hashId', 'timeSpent','Notification Shown','Notification Open'])
        
        view_data = view_data.filter(view_data.timeSpent.isNotNull())
        #view_data = view_data.filter((getHashBucketUDF(view_data.deviceId) >= 16) & (getHashBucketUDF(view_data.deviceId) <= 25))
        view_data2 = view_data.groupby(view_data.deviceId, view_data.hashId).agg(F.max(view_data.timeSpent).alias('overallTimeSpent'))
        #view_data=view_data.groupby(['deviceId','hashId','Notification Shown','Notification Open']).join(view_data2,["deviceId","hashId"],"inner")

        
        return view_data2
    except Exception as e:
        logger.warning("Error processing data: " + str(e))
        

time: 1.85 ms (started: 2023-06-19 11:29:55 +00:00)


In [120]:
from pymongo import MongoClient
from pymongo import UpdateOne

def getMongoClient():
    hosts = ['171.16.11.97','171.16.11.96','171.16.11.94']
    host = ','.join([i + ":27017" for i in hosts])
    conn_url = 'mongodb://root:superman@' + host
    client = MongoClient(conn_url)
    return client

def getMongoColl(db_name, coll_name, hosts=[]):
    host = ','.join([i + ":27017" for i in hosts])
    conn_url = 'mongodb://root:superman@' + host + '/' + db_name
    client = MongoClient(conn_url, minPoolSize=10, maxPoolSize=None)
    return client[db_name][coll_name]

def getNewsInHashIds(hashIds):
    news_coll = getMongoColl(db_name='nis-news', 
                           coll_name='News', 
                           hosts=['172.16.11.196', '172.16.11.195', '172.16.11.194'])
    
    newsMap = {}
    array = [t for t in news_coll.find({'_id' : {"$in" : hashIds}})]
    
    for i in range(len(array)):
        newsMap[array[i]['_id']] = array[i]
        
    return newsMap

def getNewsInDates(begin, end):
    news_coll = getMongoColl(db_name='nis-news', 
                           coll_name='News', 
                           hosts=['172.16.11.196', '172.16.11.195', '172.16.11.194'])
    
    newsMap = {}
    array = [t for t in news_coll.find({'createdAt' : {"$gt" : begin, "$lt" : end}})]
    
    for i in range(len(array)):
        newsMap[array[i]['_id']] = array[i]
        
    return newsMap
def getNewsData(d1, d2):
    newsMap = getNewsInDates(d1, d2)
    hashIdList = list(newsMap.keys())

    hashIdsWithFilter = []
    for h in hashIdList:
        if 'newsLanguage' in newsMap[h] and newsMap[h]['newsLanguage'] == 'english' and newsMap[h]['publishGroupList'][0]['countryCode'] == 'IN':
            hashIdsWithFilter.append(h.split('-')[0])
    
    return hashIdsWithFilter, newsMap


time: 2.17 ms (started: 2023-06-19 11:29:56 +00:00)


In [ ]:
#keeping data of 3 days for training
n_days = 40
#st_date = datetime.datetime(2023, 2, 18)
#st_date = datetime.datetime(2023, 5, 18) - datetime.timedelta(days=n_days+1)

st_date = datetime.datetime.now() - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()
#print(dates)
dates_str = [date.strftime(date_fmt) for date in dates]

# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], datetime.datetime.now())

# Taking 14Feb 

In [121]:
cd home/Gourav/Classification_model

[Errno 2] No such file or directory: 'home/Gourav/Classification_model'
/
time: 1.47 ms (started: 2023-06-19 11:30:03 +00:00)


In [122]:
n_days = 40
st_date = sampled_dates[3] - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
print(dates_str)
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])

['2023/01/04', '2023/01/05', '2023/01/06', '2023/01/07', '2023/01/08', '2023/01/09', '2023/01/10', '2023/01/11', '2023/01/12', '2023/01/13', '2023/01/14', '2023/01/15', '2023/01/16', '2023/01/17', '2023/01/18', '2023/01/19', '2023/01/20', '2023/01/21', '2023/01/22', '2023/01/23', '2023/01/24', '2023/01/25', '2023/01/26', '2023/01/27', '2023/01/28', '2023/01/29', '2023/01/30', '2023/01/31', '2023/02/01', '2023/02/02', '2023/02/03', '2023/02/04', '2023/02/05', '2023/02/06', '2023/02/07', '2023/02/08', '2023/02/09', '2023/02/10', '2023/02/11', '2023/02/12', '2023/02/13']
time: 3.07 s (started: 2023-06-19 11:30:08 +00:00)


In [123]:
datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'timeSpentFrontEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId'),data.eventTimestamp)        .groupby('deviceId', 'hashId','eventTimestamp')         .agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

data = data.filter(data.hashId.isin(hashIdsWithFilter))

time: 45 s (started: 2023-06-19 11:30:14 +00:00)


In [124]:
data

DataFrame[deviceId: string, hashId: string, eventTimestamp: bigint, overallTimeSpent: double]

time: 4.29 ms (started: 2023-06-19 11:30:59 +00:00)


In [11]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]

time: 687 µs (started: 2023-06-03 11:23:41 +00:00)


In [125]:
data=data.filter(data.overallTimeSpent<100)


time: 30.3 ms (started: 2023-06-19 11:30:59 +00:00)


In [ ]:
from datetime import datetime as dt
from pyspark.sql.functions import lit, udf, when,col
def toTokenCollection(eventTimestamp):
    dt_obj = dt.fromtimestamp(eventTimestamp)

    
    return dt_obj
#timestamp = 1545730073
from pyspark.sql.types import IntegerType
newsToTokensUdf = F.udf(toTokenCollection,DateType())

data = data.withColumn("eventTimestamp", data["eventTimestamp"].cast(IntegerType()))
data = data.withColumn("eventTimestamp", data["eventTimestamp"]/1000)

#data=data.withColumn('Event_dt',F.udf(toTokenCollection))
data = data.select('deviceId','hashId','overallTimeSpent', newsToTokensUdf('eventTimestamp').alias('Event_dt'))

In [ ]:
data.take(2)

In [ ]:
data.select('Event_dt').distinct().show(40)

# generating embedding for 30 days

In [ ]:
n_days = 30
st_date = sampled_dates[3] - datetime.timedelta(days=40+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
print(dates_str)
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])

datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'timeSpentFrontEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId'))\
        .groupby('deviceId', 'hashId') \
        .agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

data = data.filter(data.hashId.isin(hashIdsWithFilter))
data=data.filter(data.overallTimeSpent<100)

df_users=data.groupBy("deviceId").count()
df_users=df_users.withColumnRenamed("count","count_ad")
df_users=df_users.filter((df_users.count_ad< 9630) & (df_users.count_ad > 144)).select(['deviceId']).distinct()
data=data.join(df_users,["deviceId"],"inner")



def popularity(data, newsMeanMap):
    #Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data, newsMeanMap)


data_filtered = data.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))

df_notif=sqlContext.read.csv('gs://pvtrough_asia_south1/clustering/MLP_embedding_reduced', sep=',',
                         inferSchema=True, header=True)
from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,255)]
df_notif = df_notif.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))

df_notif=df_notif.withColumnRenamed("hid","hashId")

 # calculate idf_weight

N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))



# Multiply embeddings with IDF weights
#idf_weights=idf_weights.withColumnRenamed("hid","hashId")
df_result=df_embedding.join(idf_weights,["hashId"])

#df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))

df_result = df_result.select('hashId','weighted_embedding')
#df_result=df_result.withColumnRenamed("hid","hashId")
df_joined=data_filtered.join(df_result,["hashId"],"left")

#df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')

from pyspark.sql.functions import expr

grouped_df = df_joined.groupBy("deviceId").agg(*[
        expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(255)
    ])

avg_cols = [col(f"avg_{i}") for i in range(255)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/30_embeddings_users"+"/"
grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')




# Users embedding for 40 days

In [ ]:
n_days = 40
st_date = sampled_dates[3] - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
print(dates_str)
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])


hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])

datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'timeSpentFrontEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId'))\
        .groupby('deviceId', 'hashId') \
        .agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

data = data.filter(data.hashId.isin(hashIdsWithFilter))
data=data.filter(data.overallTimeSpent<100)

df_users=data.groupBy("deviceId").count()
df_users=df_users.withColumnRenamed("count","count_ad")
df_users=df_users.filter((df_users.count_ad< 9630) & (df_users.count_ad > 144)).select(['deviceId']).distinct()
data=data.join(df_users,["deviceId"],"inner")



def popularity(data, newsMeanMap):
    #Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data, newsMeanMap)


data_filtered = data.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))

df_notif=sqlContext.read.csv('gs://pvtrough_asia_south1/clustering/MLP_embedding_reduced', sep=',',
                         inferSchema=True, header=True)
from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,255)]
df_notif = df_notif.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))

df_notif=df_notif.withColumnRenamed("hid","hashId")

 # calculate idf_weight

N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))



# Multiply embeddings with IDF weights
#idf_weights=idf_weights.withColumnRenamed("hid","hashId")
df_result=df_embedding.join(idf_weights,["hashId"])

#df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))

df_result = df_result.select('hashId','weighted_embedding')
#df_result=df_result.withColumnRenamed("hid","hashId")
df_joined=data_filtered.join(df_result,["hashId"],"left")

#df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')

from pyspark.sql.functions import expr

grouped_df = df_joined.groupBy("deviceId").agg(*[
        expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(255)
    ])

avg_cols = [col(f"avg_{i}") for i in range(255)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/40_embeddings_users"+"/"
grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')


In [ ]:
grouped_df.select(['topic_avg_embedding']).show()

# embeddings for notification using text (title of news only)

In [ ]:
#@title
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback


#loading model
model_path = "output/checkpoint-12000"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=1000,output_hidden_states=True,from_tf=True)
tokenizer = RobertaTokenizer.from_pretrained(model_path)

#defining function to generate embeddings
def get_embedding(model, tokenizer, text):
    model.eval()

    
    with torch.no_grad():
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs.hidden_states
        last_layer_hidden_states = hidden_states[-1]
        embeddings = torch.mean(last_layer_hidden_states, dim=1).squeeze().cpu().numpy()

    return embeddings




In [51]:
n_days = 10
st_date = sampled_dates[3] - datetime.timedelta(days=10+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
print(dates_str)
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])


['2023/02/03', '2023/02/04', '2023/02/05', '2023/02/06', '2023/02/07', '2023/02/08', '2023/02/09', '2023/02/10', '2023/02/11', '2023/02/12', '2023/02/13']
time: 714 ms (started: 2023-06-03 10:56:19 +00:00)


In [ ]:
newsMapProcessed={}
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
# Compute document embeddings for all articles in the dataset
document_embeddings = {}
for hId in tqdm(newsMap):
    newsid = hId.split('-')[0]
    #print(h)
    #newsMapProcessed[newsid] = {}
    #newsMapProcessed[newsid]['title'] = newsMap[hId]['title']
    #newsMapProcessed[newsid]['content'] = newsMap[hId]['content']
    
    #newsMapProcessed[newsid]['features'] = newsMapProcessed[h]['title'] + newsMapProcessed[h]['content']
    text=newsMap[hId]['title']
    document_embeddings[newsid] = get_embedding(model, tokenizer, text)


# Regressor for training 

In [152]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]

time: 872 µs (started: 2023-06-19 12:33:12 +00:00)


In [153]:
n_days = 10
st_date = sampled_dates[3] - datetime.timedelta(days=10+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
print(dates_str)
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[3])


['2023/02/03', '2023/02/04', '2023/02/05', '2023/02/06', '2023/02/07', '2023/02/08', '2023/02/09', '2023/02/10', '2023/02/11', '2023/02/12', '2023/02/13']
time: 791 ms (started: 2023-06-19 12:33:16 +00:00)


In [ ]:
datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'timeSpentFrontEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
data = data.filter(data.categories.isNotNull())
from pyspark.sql.functions import explode

#data=data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'),data.notificationOpened,explode(data.categories)).withColumnRenamed("col","category")
data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId')).groupby('deviceId', 'hashId').agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

data = data.filter(data.hashId.isin(hashIdsWithFilter))
#data=data.filter(data.overallTimeSpent<100)

#data=data.na.fill(value=0,subset=["notificationOpened"])
#data=data.na.fill(value=0,subset=["notificationOpened"])



In [155]:
data

DataFrame[deviceId: string, hashId: string, overallTimeSpent: double]

time: 5.69 ms (started: 2023-06-19 12:33:37 +00:00)


In [ ]:
df_notif = sqlContext.read.csv('gs://pvtrough_asia_south1/clustering/df_embedding_LM_large.csv', sep=',',
                         inferSchema=True, header=True)
from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,768)]
df_notif = df_notif.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))

df_notif=df_notif.withColumnRenamed("hid","hashId")

In [157]:
df_notif

DataFrame[hashId: string, embedding: array<double>]

time: 3.8 ms (started: 2023-06-19 12:34:19 +00:00)


In [ ]:
df_user_embedd.show()

In [158]:
path_vec = "gs://pvtrough_asia_south1/clustering/30_embeddings_users"+"/"
df_devices=sqlContext.read.parquet(path_vec)

time: 292 ms (started: 2023-06-19 12:34:38 +00:00)


In [159]:
df_devices

DataFrame[deviceId: string, topic_avg_embedding: array<double>]

time: 3.82 ms (started: 2023-06-19 12:34:42 +00:00)


In [160]:
df_train=data.join(df_notif,["hashId"],"inner").join(df_devices,['deviceId'],"inner").select(['deviceId','hashId','overallTimeSpent','embedding','topic_avg_embedding'])


time: 48.6 ms (started: 2023-06-19 12:34:45 +00:00)


In [161]:
from pyspark.ml.functions import array_to_vector
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback
from pyspark.sql.functions import array, col
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains, size, col
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import expr, udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from sklearn.neural_network import MLPRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator


time: 3.12 ms (started: 2023-06-19 12:34:50 +00:00)


In [162]:
from pyspark.sql.functions import col, concat

train_data = df_train.withColumn('embedding_features', 
                                            concat(col('topic_avg_embedding'), col('embedding')))\
                                .select('embedding_features', 'overallTimeSpent')

train_data=train_data.select(array_to_vector('embedding_features').alias('vec_embedding'),'overallTimeSpent')#.collect()

time: 45.9 ms (started: 2023-06-19 12:34:52 +00:00)


# Training using Linear regression

In [ ]:

from pyspark.ml.regression import LinearRegression


lr = LinearRegression(featuresCol = 'vec_embedding', labelCol='overallTimeSpent', maxIter=100, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

# Prediction using Model

In [ ]:
#test_df is data of next 7 days news embedding and 40 days users embdding

In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

In [ ]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

# Using Decision tree for regression

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressordt = DecisionTreeRegressor(featuresCol ='vec_embedding', labelCol = 'overallTimeSpent')
dt_model = dt.fit(train_data)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="overallTimeSpent", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# Using GBT regressor

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'vec_embedding', labelCol = 'overallTimeSpent', maxIter=10)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'overallTimeSpent', 'features').show(5)

In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="MV", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# Using XGB regressor

In [164]:
from xgboost.spark import SparkXGBRegressor
spark_reg_estimator = SparkXGBRegressor(
  features_col="vec_embedding",
  label_col="overallTimeSpent",
  num_workers=4,
)



time: 2.08 ms (started: 2023-06-19 12:35:40 +00:00)


In [ ]:
xgb_regressor_model = spark_reg_estimator.fit(train_data)

In [ ]:
transformed_test_spark_dataframe = spark_reg_estimator.predict(test_spark_dataframe)

In [ ]:
from xgboost.spark import SparkXGBRegressor
#spark = SparkSession.builder.getOrCreate()


label_name = "overallTimeSpent"

feature_names = "vec_embedding"

regressor = SparkXGBRegressor(
  features_col=feature_names,
  label_col=label_name,
  num_workers=2,
  use_gpu=True,
)

model = regressor.fit(train_data)

predict_df = model.transform(test_df)
predict_df.show()


In [66]:
df_train.count()

484685207

time: 1min 29s (started: 2023-06-12 10:51:46 +00:00)


In [67]:
user_embedd=data.select(['deviceId']).distinct().join(df_devices,['deviceId'],'inner')

time: 23.1 ms (started: 2023-06-12 10:53:16 +00:00)


In [68]:
user_embedd.write.parquet(path="gs://pvtrough_asia_south1/clustering/30_embeddings_users_v2"+"/", mode='overwrite')

time: 1min 12s (started: 2023-06-12 10:53:16 +00:00)


In [69]:
user_embedd.count()

1358084

time: 57.8 s (started: 2023-06-12 10:54:28 +00:00)


In [ ]:
n_splits = 20
  
each_len = user_embedd.count() // n_splits
  
copy_df = user_embedd
hhhhg=[]
i = 0
while i < n_splits:
  
    temp_df = copy_df.limit(each_len)
  
    copy_df = copy_df.subtract(temp_df)
    #temp_df.cache()
    #temp_df.show(truncate=False)
    temp=temp_df.select('topic_avg_embedding').rdd.flatMap(lambda x: x).collect()
    
    #user_weight=torch.FloatTensor(hhhhg)
    hhhhg.extend(temp)
    print(i)
    #path_vec = "gs://pvtrough_asia_south1/clustering/30_embeddings_users_v2"+str(i)+"/"
    #temp_df.write.parquet(path=path_vec, mode='overwrite')
    #print(path_vec)

    i += 1
    

In [70]:
df_temp=user_embedd.sample(0.025)

time: 5.67 ms (started: 2023-06-12 10:55:26 +00:00)


In [71]:
df_temp

DataFrame[deviceId: string, topic_avg_embedding: array<double>]

time: 4.88 ms (started: 2023-06-12 10:55:26 +00:00)


In [72]:
hhhhg=df_temp.select('topic_avg_embedding').rdd.flatMap(lambda x: x).collect()


time: 1min 5s (started: 2023-06-12 10:55:26 +00:00)


In [ ]:
hhhhg[0]

In [ ]:
hh[0]

In [73]:
notif_embedd=data.select(['hashId']).distinct().join(df_notif,['hashId'],'inner')

time: 27.2 ms (started: 2023-06-12 10:56:31 +00:00)


In [ ]:
notif_embedd.count()

In [75]:
hh=notif_embedd.select('embedding').rdd.flatMap(lambda x: x).collect()


time: 58.6 s (started: 2023-06-12 10:57:28 +00:00)


In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as F

time: 1.31 s (started: 2023-06-12 10:58:26 +00:00)


In [ ]:
n_splits = 20
i = 0
user_weight_dict={}
while i < n_splits:
    path_vec = "gs://pvtrough_asia_south1/clustering/30_embeddings_users_v2"+str(i)+"/"
    user_embedd=sqlContext.read.parquet(path_vec)
    hhhhg=user_embedd.select("topic_avg_embedding").rdd.flatMap(lambda x: x).collect()
    user_weight=torch.FloatTensor(hhhhg)
    
    

    

In [ ]:
user_embedd.count()

In [77]:
user_weight=torch.FloatTensor(hhhhg)
user_embedding = nn.Embedding.from_pretrained(user_weight)

time: 1.65 s (started: 2023-06-12 10:59:41 +00:00)


In [78]:
notif_weight=torch.FloatTensor(hh)
notification_embedding = nn.Embedding.from_pretrained(notif_weight)

time: 139 ms (started: 2023-06-12 10:59:59 +00:00)


In [67]:
input = torch.LongTensor([1])

time: 641 µs (started: 2023-06-05 12:09:52 +00:00)


In [69]:
input

tensor([1])

time: 2.73 ms (started: 2023-06-05 12:11:31 +00:00)


In [82]:
notification_embedding(torch.LongTensor([5]))[0].pow(2).sum()

tensor(128.9333)

time: 3.84 ms (started: 2023-06-05 12:41:43 +00:00)


In [91]:
notif_weight[5].pow(2).sum()

tensor(128.9333)

time: 3.44 ms (started: 2023-06-05 12:45:26 +00:00)


In [93]:
len((notif_weight*user_weight).sum(1))

2851

time: 3.02 ms (started: 2023-06-05 12:46:41 +00:00)


In [83]:
notification_embedding

Embedding(2851, 768)

time: 2.18 ms (started: 2023-06-05 12:42:01 +00:00)


In [80]:
notification_embedding(torch.LongTensor([5]))[0]

tensor([-1.5108e-02,  1.7124e-02,  6.4353e-02, -6.5654e-02, -2.4161e-01,
        -2.6214e-02,  5.6563e-03,  8.6963e-02,  1.6884e-01,  3.3735e-02,
        -1.4390e-01, -1.3412e-01, -5.2816e-03, -6.9174e-02,  1.3551e-01,
        -3.0892e-01,  1.5388e-01,  7.7667e-02,  9.2706e-03,  4.9212e-02,
         6.1749e-02,  2.1046e-01, -1.1446e-01, -2.7498e-02, -2.2766e-02,
         3.8531e-02,  4.8362e-02,  6.5857e-03, -7.1668e-02,  7.7646e-03,
        -1.1520e-01,  2.0958e-02, -8.4604e-03, -4.0875e-04, -1.0900e-01,
        -4.7042e-02,  1.4840e-01,  4.2106e-03,  3.7527e-01,  7.4570e-02,
         2.6634e-01, -1.2184e-02, -7.3225e-02,  5.4505e-02, -3.8288e-03,
        -8.5135e-03, -8.3278e-02, -1.2030e-02,  3.9430e-02,  8.7702e-03,
        -1.6854e-01,  6.7730e-02,  1.8065e-01,  6.3362e-02, -7.3808e-02,
        -2.0488e-02,  2.8329e-02, -1.6032e-01,  6.9426e-02, -1.0553e-01,
         2.5786e-02, -3.3725e-01, -1.2825e-01,  5.0514e-02, -3.6329e-02,
        -4.0595e-02, -1.0010e-01, -3.7579e-02, -5.2

time: 9.17 ms (started: 2023-06-05 12:40:36 +00:00)


In [30]:
len(notif_weight)

2851

time: 2.13 ms (started: 2023-06-05 07:09:49 +00:00)


In [40]:
len(hhhhg)

68677

time: 2.41 ms (started: 2023-06-05 10:15:02 +00:00)


In [ ]:
hhhhg[68676]

In [48]:
type(hhhhg)

list

time: 2.4 ms (started: 2023-06-05 10:18:05 +00:00)


In [49]:
any(elem is None for elem in hhhhg)


False

time: 6.14 ms (started: 2023-06-05 10:19:07 +00:00)


In [64]:
user_embedd_9098=user_embedd.coalesce(20)

time: 6.64 ms (started: 2023-06-05 11:06:18 +00:00)


In [66]:
user_embedd_9098.write.partitionBy("deviceId").parquet("gs://pvtrough_asia_south1/clustering/30_embeddings_users_v2"+str(678)+"/",mode='overwrite')


Py4JJavaError: An error occurred while calling o5496.parquet.
: java.util.NoSuchElementException: None.get
	at scala.None$.get(Option.scala:529)
	at scala.None$.get(Option.scala:527)
	at org.apache.spark.sql.execution.datasources.BasicWriteJobStatsTracker$.metrics(BasicWriteStatsTracker.scala:175)
	at org.apache.spark.sql.execution.command.DataWritingCommand.metrics(DataWritingCommand.scala:51)
	at org.apache.spark.sql.execution.command.DataWritingCommand.metrics$(DataWritingCommand.scala:51)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.metrics$lzycompute(InsertIntoHadoopFsRelationCommand.scala:49)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.metrics(InsertIntoHadoopFsRelationCommand.scala:49)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.metrics$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.metrics(commands.scala:104)
	at org.apache.spark.sql.e

time: 245 ms (started: 2023-06-05 11:06:33 +00:00)
